### GSI summer student program 

Started this notebook on 09/08/2024. First goal is to perform a balistic correction on the energies measured by the 16 strip detector, in order to do it I extracted data from the go4 histogram to perform a quadratic fit on energies vs. strip position for each strip.

In [1]:
# importing the useful packages
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from scipy import optimize

# defining directories 
work_dir = os.getcwd()
data_dir = os.path.join(work_dir, "Uncalib_energy")


print("work dir is %s" % work_dir)
print("data dir is %s" % data_dir)

work dir is d:\GSI\SHIP
data dir is d:\GSI\SHIP\Uncalib_energy


In [2]:
# reading data
filename = "16 Strip Position Corrected Energy 01.hdat"
strip_number = "01"
filepath = os.path.join(data_dir, filename)

my_df = pd.read_csv(filepath, sep="\t", header=1) 
my_df.columns

Index(['# X ', 'Y ', 'Z ', 'Content'], dtype='object')

In [ ]:

# Extract x, y, and counts
position = my_df["# X "].to_numpy()
energy = my_df["Y "].to_numpy()
content = my_df['Content'].to_numpy()

# selecting only the relevant bins
index = np.where(content!=0)
selected_energy = energy[index]
selected_position = position[index]
selected_content = content[index]
print(index[0])

# Determine the number of unique bins in x and y
x_bins = np.unique(selected_position)
y_bins = np.unique(selected_energy)

# Original number of bins
original_x_bins = len(x_bins)
original_y_bins = len(y_bins)

# Define the desired number of bins
new_x_bins = 26  # Modify as needed
new_y_bins = 200  # Modify as needed

# Calculate the new bin edges
x_edges = np.linspace(x_bins.min(), x_bins.max(), new_x_bins + 1)
y_edges = np.linspace(y_bins.min(), y_bins.max(), new_y_bins + 1)

# Re-bin the counts
counts_2d, _, _ = np.histogram2d(selected_position, selected_energy, bins=[x_edges, y_edges], weights=selected_content)

# Plot the re-binned 2D histogram using imshow
plt.figure(figsize=(10,6))
plt.imshow(counts_2d.T, origin='lower', extent=[x_bins.min(), x_bins.max(), y_bins.min(), y_bins.max()], aspect='auto', cmap='twilight')
plt.colorbar(label='Counts')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Re-binned 2D Histogram')
plt.show()

indices = np.where(counts_2d > 10)

# Extract the bin edges corresponding to these indices
x_coords = (x_edges[indices[0]] + x_edges[indices[0] + 1]) / 2
y_coords = (y_edges[indices[1]] + y_edges[indices[1] + 1]) / 2

xx = []
yy = []

# Print or store the coordinates
for x, y in zip(x_coords, y_coords):
    print(f"X: {x}, Y: {y}")
    xx.append(x)
    yy.append(y)

yy = np.array(yy)
xx = np.array(xx)

plt.scatter(xx,yy)


In [ ]:

yy = np.array(yy)
xx = np.array(xx)

fig3=plt.figure(figsize=(10,8))
plt.scatter(xx,yy, label='Data')

cond1 = yy<3500 # condition for first curve
cond2_a = yy>3500 # condition for second curve
cond2_b = yy<3700 # condition for second curve
cond3 = yy>3700 # condition for third curve

# first curve
index1 = np.where(cond1)
yy1 = yy[index1]
xx1 = xx[index1]
print(yy1)

# second curve
index2 = np.where(np.logical_and(cond2_a, cond2_b))
yy2 = yy[index2]
xx2 = xx[index2]

# third curve
index3 = np.where(cond3)
yy3 = yy[index3]
xx3 = xx[index3]

In [ ]:
# trying to fit all three


popt, pcov = optimize.curve_fit(parabola, xx1, yy1)
print("a1 =", popt[0], "+/-", pcov[0,0]**0.5)
print("b1 =", popt[1], "+/-", pcov[1,1]**0.5)
print("c1 =", popt[2], "+/-", pcov[2,2]**0.5)


popt2, pcov2 = optimize.curve_fit(parabola, xx2, yy2)
print("a2 =", popt2[0], "+/-", pcov2[0,0]**0.5)
print("b2 =", popt2[1], "+/-", pcov2[1,1]**0.5)
print("c2 =", popt2[2], "+/-", pcov2[2,2]**0.5)


popt3, pcov3 = optimize.curve_fit(parabola, xx3, yy3)
print("a3 =", popt3[0], "+/-", pcov3[0,0]**0.5)
print("b3 =", popt3[1], "+/-", pcov3[1,1]**0.5)
print("c3 =", popt3[2], "+/-", pcov3[2,2]**0.5)

# Plotting the fit for the first curve

fig3=plt.figure(figsize=(10,8))
plt.scatter(xx,yy, label='Data')
plt.plot(xx1, parabola(xx1, *popt), "r-", label='fit')

plt.xlabel("Strip position", fontsize=15)
plt.ylabel("Channel", fontsize=15)
plt.title("Scatter plot of Energy vs Calib Position strip %s" % strip_number, fontsize=20)

# plotting the fit for the second curve
plt.plot(xx2, parabola(xx2, *popt2), "r-")

# plotting the fit for the third curve
plt.plot(xx3, parabola(xx3, *popt3), "r-")
plt.legend()
plt.grid(True)


In [ ]:
# Writing fit results in text file

output_name = 'prova_fit_results'
results_dir = os.path.join(work_dir, 'results')
output_filename = os.path.join(results_dir, output_name)

text=open(output_filename+strip_number+".dat", "w")

print("# Strip %s fit results" % strip_number, '\n', file=text)
print(popt[0], popt[1], popt[2] - np.min(yy1), file=text)
print(popt2[0], popt2[1], popt2[2] - np.min(yy2), file=text)
print(popt3[0], popt3[1], popt3[2] - np.min(yy3), file=text)

text.close()

In [ ]:
# Plotting the result of the calibration for the first curve

fig3=plt.figure(figsize=(10,8))
plt.scatter(xx1, yy1-parabola(xx1, *popt) + np.min(yy1), label='fit')

plt.xlabel("Strip position", fontsize=15)
plt.ylabel("Channel", fontsize=15)
plt.title("Scatter plot of Energy vs Calib Position strip %s" % strip_number, fontsize=20)

# plotting the fit for the second curve
plt.scatter(xx2, yy2-parabola(xx2, *popt2)+np.min(yy2))

# plotting the fit for the third curve
plt.scatter(xx3, yy3-parabola(xx3, *popt3)+np.min(yy3))
plt.grid(True)


for jj in range(len(selected_energy)-1):
    if selected_energy[jj]<3500:
        selected_energy[jj] = selected_energy[jj] - parabola(selected_position[jj], *popt) + np.min(yy1)
    elif selected_energy[jj]>3700:
        selected_energy[jj] = selected_energy[jj] - parabola(selected_position[jj], *popt3) + np.min(yy2)
    else:
        selected_energy[jj] = selected_energy[jj] - parabola(selected_position[jj], *popt2) + np.min(yy3)


# Determine the number of unique bins in x and y
x_bins = np.unique(selected_position)
y_bins = np.unique(selected_energy)

# Original number of bins
original_x_bins = len(x_bins)
original_y_bins = len(y_bins)

# Define the desired number of bins
new_x_bins = 26  # Modify as needed
new_y_bins = 200  # Modify as needed

# Calculate the new bin edges
x_edges = np.linspace(x_bins.min(), x_bins.max(), new_x_bins + 1)
y_edges = np.linspace(y_bins.min(), y_bins.max(), new_y_bins + 1)

# Re-bin the counts
counts_2d, _, _ = np.histogram2d(selected_position, selected_energy, bins=[x_edges, y_edges], weights=selected_content)


plt.figure(figsize=(10,6))
plt.imshow(counts_2d.T, origin='lower', extent=[x_bins.min(), x_bins.max(), y_bins.min(), y_bins.max()], aspect='auto', cmap='twilight')
plt.colorbar(label='Counts')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Re-binned 2D Histogram')
plt.show()

In [ ]:
def parabola(x, a1, b1, c1): 
    par1 = a1*x**2 + b1*x + c1

    return par1